# Presentation de l'entrainement PMC

Nous allons ici presenté le code et sont fonctionnement via des interaction direct avec les differents dossiers et fichiers de training

voila le code principal qui nous permet de faire des models sur nos données:


d'abord l'importation les lib et la definition des code de notre librairy


In [ ]:
import os 
from PIL import Image
import numpy as np
import ctypes
import sys
import create_listTest
import trainer
import pickle
import subprocess
from IPython.display import display
import matplotlib.pyplot as plt

dirname = os.getcwd()
chemin = os.path.join(dirname,"..", "Test_image")
create_listTest.allcolors(chemin)
create_listTest.expected_image(chemin)

# Get the directory of the current script
current_dir = os.getcwd()

# Append the path to the DLL to the system's search path
dll_path = os.path.join(current_dir, 'perceptron_multi_couche.dll')
sys.path.append(dll_path)

# Load the DLL
mlp_dll = ctypes.cdll.LoadLibrary(dll_path)

# Définition des types de données
mlp_dll.createMLP.restype = ctypes.c_void_p
mlp_dll.createMLP.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_int]
mlp_dll.deleteMLP.argtypes = [ctypes.c_void_p]




# Modifier les si vous voulez produire des resultat différent !
les paramètres d'apprentissages


In [ ]:
# Définition de la structure du MLP et paramètres

npl = np.array([1875,32,3], dtype=np.int32)
num_epochs = 100000
learning_rate = 0.006545
mlp_ptr = mlp_dll.createMLP(npl.ctypes.data_as(ctypes.POINTER(ctypes.c_int)), npl.size)

definition de la base de train et entrainement

In [ ]:
data_dir = os.path.join(current_dir)
train_inputs = np.array(create_listTest.allcolors(os.path.join(data_dir, '..', 'Train_image')))
train_expected_outputs = np.array(create_listTest.expected_image(os.path.join(data_dir, '..', 'Train_image')))
print(train_inputs.shape)
# Entraînement du MLP sur un nombre spécifique d'époques
samples_inputs = train_inputs
samples_expected_outputs = train_expected_outputs

command = ["python", "-c", f"import trainer; trainer.training({num_epochs}, {learning_rate}, {samples_inputs}, {samples_expected_outputs}, {mlp_ptr})"]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, universal_newlines=True)

# Afficher la sortie du terminal dans Jupyter Notebook
print(result.stdout)

mlp_ptr = trainer.training(num_epochs, learning_rate,samples_inputs,samples_expected_outputs,mlp_ptr)



les image que qui n'ont pas de format RGB ne sont aps traité ici on traite des image avec 3 valeur par pixel 3 x 25 x 25 = 1875 notre nombre d'entrée

Puis prediction de d'image via la base de test !

In [ ]:
# Charger les données de test
test_inputs = np.array(create_listTest.allcolors(os.path.join(data_dir, '..', 'Test_image')))
test_expected_outputs = np.array(create_listTest.expected_image(os.path.join(data_dir, '..', 'Test_image')))

# Charger les images d'entrée et les étiquettes attendues
input_images = test_inputs
expected_outputs = test_expected_outputs

# Appeler la fonction predict_list
predicted_outputs = trainer.predict(mlp_ptr, test_inputs)

# Obtenir l'indice de la classe prédite pour chaque prédiction
# Obtenir l'indice de la classe attendue pour chaque étiquette
predicted_classes = np.argmax(predicted_outputs, axis=1)
expected_classes = np.argmax(test_expected_outputs, axis=1)

diffs = predicted_classes - expected_classes

# Tracer un graphique des différences
plt.plot(diffs)
plt.title("Différences entre les classes prédites et les classes attendues")
plt.xlabel("Exemples d'entrée")
plt.ylabel("Différence entre la classe prédite et la classe attendue")
plt.show()

# Afficher les classes prédites et les classes attendues côte à côte
print("Prédictions du MLP :")
for i in range(len(predicted_classes)):
    predicted_class = predicted_classes[i]
    expected_class = expected_classes[i]
    print(f"Exemple {i+1} - Prédiction : {predicted_class}, Attendu : {expected_class}")
    print(f"Sortie du MLP : {predicted_outputs[i]}")